In [1]:
import tensorflow as tf

2022-05-03 12:24:33.598842: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-03 12:24:33.607847: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(tf.__version__)

2.7.0


In [3]:
import requests
import pandas as pd
import numpy as np

In [5]:
new_model = tf.keras.models.load_model('rede1.h5')

2022-05-03 12:32:49.961845: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-03 12:32:49.988192: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2022-05-03 12:32:50.349927: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-03 12:33:00.002124: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6480000 exceeds 10% of free system memory.
2022-05-03 12:33:00.244553: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 6480000 exceeds 10% of free system memory.
2022-05-03 12:33:00

In [6]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 900)               1620900   
                                                                 
 dense_1 (Dense)             (None, 10)                9010      
                                                                 
Total params: 1,629,910
Trainable params: 1,629,910
Non-trainable params: 0
_________________________________________________________________


In [ ]:
codigoGrupo = "Neg-100"
data = "11"
codigoRegra = "t11s17v"

url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaos/obtemConjuntoTreino?idPeriodo={}&codigoGrupoAcao={}&codigoRegraProjecao={}".format(codigoPeriodo,codigoGrupo,codigoRegra)